In [2]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [3]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [4]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "originalFileJPG":"",
    "originalFileGrp":[],
    "destFolder":"",
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "dest":{
      "rgb":"",
      "ndvi":"",
      "thumb":"",
    },
    "info":{
      "course":"",
      "area":"",
      "desc":"",
      "alt":""
    },
    "label":[],
    "annotation":[],
    "labelBy":""
  }

# Remove Drive letter with root path
  dslabel['originalFileJPG'] = img_path[3:]
  dslabel['originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:.10f}".format(coords[0]).replace(".","") + "_{:.10f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [5]:
def getNDVIimg(filesGrp):


  with rasterio.open([ x for x in filesGrp if ('_MS_R' in x) ][0]) as src:
      band_red = src.read(1).astype(float)/65536.

  with rasterio.open([ x for x in filesGrp if ('_MS_NIR' in x) ][0]) as src:
      band_nir = src.read(1).astype(float)/65536.

  np.seterr(divide='ignore', invalid='ignore')

  # Calculate NDVI
  ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir.astype(float) + band_red.astype(float))

  ndvi -= ndvi.min() # ensure the minimal value is 0.0
  ndvi /= ndvi.max() # maximum value in image is now 1.0

  cm = plt.cm.get_cmap('RdYlGn')
  ndvi_cm = cm(ndvi)

  ndvi_img = cv2.normalize(ndvi_cm[:,:,:3], None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)

  return ndvi_img

In [6]:
def getalignedRGB(filesGrp_, ndvi_img):
  RGBimg = np.asarray(Image.open([x for x in filesGrp_ if 'JPG' in x][0]))

  # RGBimg = np.asarray(Image.open(filesGrp_[0]).resize((ndvi_img.shape[1],ndvi_img.shape[0])))
  # features0 = FeatureExtraction(RGBimg)
  # features1 = FeatureExtraction(ndvi_img)

  # matches = feature_matching(features0, features1)

  # print('{} number of matched has been found! in {}'.format(len(matches), filesGrp_[0]))

  # if len(matches) < 10:
  #    raise Exception("Not Enough Matches")

  # H, _ = cv2.findHomography( features0.matched_pts, features1.matched_pts, cv2.RANSAC, 5.0)

  H = np.array([[ 5.93209530e-01, -8.35074813e-03, -2.21474434e+02],
       [ 1.32198817e-02,  5.96460559e-01, -2.01042419e+02],
       [ 1.58520522e-06,  1.69424781e-06,  1.00000000e+00]])

  h, w, c = ndvi_img.shape
  RGBimg_tuned = cv2.warpPerspective(RGBimg, H, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

  return RGBimg_tuned 

In [7]:
img_folder = r'D:\NIA_2023_52_잔디\10_원시데이터(3사_분류대상)'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [8]:
course_id = course_ids.get('청도')
course_id

'MGC002'

In [9]:
img_folders_candidate =glob.glob(os.path.join(img_folder,'*'))
img_folders_candidate

['D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0817_거창클럽디_오전구름많이오후구름반햇빛반_데이터 1차 분류본_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0822_시흥아세코밸리_비온직후맑은날_데이터 1차 분류본_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0825_클럽디속리산_맑고뭉게구름_데이터1차분류_M3M_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0826_춘천오너스_맑고구름조금미세먼지_데이터 1차 분류_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0902_시흥아세코밸리_맑은날_구름조금_데이터 1차 분류_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0906_거창클럽디_맑은날_구름조금_데이터 1차 분류_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0908_포항CC_데이터 1차 분류_완',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0909_중문골프클럽_데이터 1차 분류_완']

In [10]:
img_folder = img_folders_candidate[6]
img_folder

'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완'

In [11]:
files = glob.glob(os.path.join(img_folder,'**/DJI_*.JPG'), recursive=True)

len(files)

9893

In [12]:
original_folder = []

for file_ in files:
  path_ = os.path.normpath(file_)
  paths_ = path_.split(os.sep)
  path_ = (os.sep).join(paths_[0:5])

  if path_ not in original_folder:
    original_folder.append(path_)

In [13]:
original_folder

['D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크2H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크4H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴1H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴2H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴3H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴4H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴5H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴6H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴7H',
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907

In [19]:
target_path = original_folder[6]
target_files = glob.glob(os.path.join(target_path,'**/DJI_*.JPG'), recursive=True)
len(target_files), target_path


(780,
 'D:\\NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_마운틴4H')

In [20]:
course = list(filter(lambda y: y in img_folder, course_list))[0]
course_id = course_ids.get(course)

path_ = os.path.normpath(target_path).split(os.sep)

target_date = ('').join(path_[-2].split('_')[:2])
area = path_[-1]

course_id, target_date, area



('MGC002', '20230907', '청도그레이스_마운틴4H')

In [21]:
file_ = target_files[0]

path = os.path.normpath(file_)
info = path.split(os.sep)
root_index = [idx for idx, s in enumerate(info) if 'NIA_2023_52_잔디' in s][0]
area = info[root_index + 3]

if len(info[root_index + 5].split('_')) ==3 :
  desc = info[root_index + 5].split('_')[-1] + '_0'
else:
  desc = "미기재" + '_0'


out_folder,course_id,target_date,area,desc

('D:\\ToAWS2', 'MGC002', '20230907', '청도그레이스_마운틴3H', '패취류,섬머패취,켄터키,티_0')

In [22]:
InfoJsonArray = []
target_folder = []

for idx, file_ in enumerate(target_files):

  path = os.path.normpath(file_)
  info = path.split(os.sep)
  root_index = [idx for idx, s in enumerate(info) if 'NIA_2023_52_잔디' in s][0]
  area = info[root_index + 3]
  
  if len(info[root_index + 5].split('_')) ==3 :
    desc = info[root_index + 5].split('_')[-1] + '_0'
  else:
    desc = "미기재" + '_0'

  target_path = ('/').join([target_date,area,desc])

  while target_folder.count(target_path)>500:
    desc = desc.split('_')[0] + '_'+str(int(desc.split('_')[-1]) + 1)
    target_path = ('/').join([target_date,area,desc])

  target_folder.append(target_path)

  alt =  info[root_index + 4].split('_')[-1]


  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(os.path.split(file_)[1].split('_')[0]+'_'+ 
    os.path.split(file_)[1].split('_')[1][:-3]+"***_" + os.path.split(file_)[1].split('_')[2]) + '*'), recursive=True)
  
  file_info = getInfo(filesGrp, file_)

  file_info['info']['course'] = course
  file_info['info']['area'] = area
  file_info['info']['alt'] = alt
  file_info['info']['desc'] = desc
  file_info['info']['date'] = target_date
  file_info['destFolder'] = target_path

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'rgb')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi')

  if not os.path.exists(path_):
    os.makedirs(path_)

  path_ = os.path.join(out_folder,course_id,target_date,area,desc,'thumb')

  if not os.path.exists(path_):
    os.makedirs(path_)


  try:
    ndvi_img = getNDVIimg(filesGrp)
    rgb_img = getalignedRGB(filesGrp, ndvi_img)
    resized_ndvi = cv2.resize(ndvi_img, dsize=(648,486), interpolation=cv2.INTER_CUBIC)
    save_name = os.path.join(out_folder,course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )
    im = Image.fromarray(resized_ndvi)
    im.save(save_name)
    file_info['dest']['ndvi'] = os.path.join(course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )

    resized_rgb = cv2.resize(rgb_img , dsize=(2592,1944), interpolation=cv2.INTER_CUBIC)
    save_name = os.path.join(out_folder,course_id,target_date,area,desc,'rgb','rgb{}.JPG'.format( file_info['id']) )
    im = Image.fromarray(resized_rgb)
    im.save(save_name)
    file_info['dest']['rgb'] = os.path.join(course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )


    thumb_rgb = cv2.resize(rgb_img , dsize=(324,243), interpolation=cv2.INTER_CUBIC)
    save_name = os.path.join(out_folder,course_id,target_date,area,desc,'thumb','thumbrgb{}.JPG'.format( file_info['id']) )
    im = Image.fromarray(thumb_rgb)
    im.save(save_name)
    file_info['dest']['thumb'] = os.path.join(course_id,target_date,area,desc,'ndvi','ndvi{}.JPG'.format( file_info['id']) )

    InfoJsonArray.append( file_info)
    print('{} out of {} files Completed'.format(idx, len(target_files)), end='\r')
    print(save_name)
  except Exception as e:
    print("{} file Error".format(idx) + str(e))
    # logfile.write("{} file Error".format(idx) +'('+file_+')'+ str(e))
  finally:
    continue
  
  

save_name = os.path.join(out_folder,course_id,target_date,area, 'data'+target_date+area+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)


  

C:\Users\yc463\AppData\Local\Temp\ipykernel_41048\891679340.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('RdYlGn')


C:\Users\yc463\AppData\Local\Temp\ipykernel_41048\1506130912.py:2: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MpoImageFile'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  RGBimg = np.asarray(Image.open([x for x in filesGrp_ if 'JPG' in x][0]))


248 file ErrorOpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'warpPerspective'
> Overload resolution failed:
>  - src data type = 17 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'



Gather all data json files from Output Folder

In [27]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['D:\\ToAWS\\MGC001\\20230816\\1.태백3H\\data202308161.태백3H.json',
 'D:\\ToAWS\\MGC001\\20230816\\2.태백5H\\data202308162.태백5H.json',
 'D:\\ToAWS\\MGC001\\20230816\\3.동해1H\\data202308163.동해1H.json',
 'D:\\ToAWS\\MGC001\\20230816\\4.동해2H\\data202308164.동해2H.json',
 'D:\\ToAWS\\MGC001\\20230816\\5.동해3H\\data202308165.동해3H.json',
 'D:\\ToAWS\\MGC001\\20230816\\6.태백1H\\data202308166.태백1H.json']

In [28]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r") as _json:
    dataJson= json.load(_json)
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [29]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(totalPhotoJson, final , ensure_ascii=False)

In [30]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [31]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)

In [32]:
len(totalPhotoJson)

9184